# Exploiting SetupComplete.cmd CVE-2019-1378
Detects exploitation attempt of privilege escalation vulnerability via SetupComplete.cmd and PartnerSetupComplete.cmd decribed in CVE-2019-1378

## Rule Content
```
- title: Exploiting SetupComplete.cmd CVE-2019-1378
  id: 1c373b6d-76ce-4553-997d-8c1da9a6b5f5
  status: experimental
  description: Detects exploitation attempt of privilege escalation vulnerability
    via SetupComplete.cmd and PartnerSetupComplete.cmd decribed in CVE-2019-1378
  references:
  - https://www.embercybersecurity.com/blog/cve-2019-1378-exploiting-an-access-control-privilege-escalation-vulnerability-in-windows-10-update-assistant-wua
  author: Florian Roth
  date: 2019/11/15
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1055
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentCommandLine:
      - '*\cmd.exe /c C:\Windows\Setup\Scripts\SetupComplete.cmd'
      - '*\cmd.exe /c C:\Windows\Setup\Scripts\PartnerSetupComplete.cmd'
    filter:
      Image:
      - C:\Windows\System32\\*
      - C:\Windows\SysWOW64\\*
      - C:\Windows\WinSxS\\*
      - C:\Windows\Setup\\*
    condition: selection and not filter
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_command_line.keyword:(*\\cmd.exe\ \/c\ C\:\\Windows\\Setup\\Scripts\\SetupComplete.cmd OR *\\cmd.exe\ \/c\ C\:\\Windows\\Setup\\Scripts\\PartnerSetupComplete.cmd) AND (NOT (process_path.keyword:(C\:\\Windows\\System32\\* OR C\:\\Windows\\SysWOW64\\* OR C\:\\Windows\\WinSxS\\* OR C\:\\Windows\\Setup\\*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()